In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from ollama import Client
from IPython.display import display_markdown, display_latex

from src.agentic.tool.tool import ToolRegistry, ToolCallProcessor
from functions import summation, multiplication

### Connect to local Ollama_server

In [29]:
client = Client(host="http://localhost:11434")

model_name = "qwen2.5:7b"

# Pull a model
result = client.pull(model=model_name)
result

{'status': 'success'}

### Registry functions

In [30]:
# Instanciar o registro de ferramentas
tool_registry = ToolRegistry()

# Registrar as funções
tool_registry.register("summation", summation)
tool_registry.register("multiplication", multiplication)

tools_schemas_string = tool_registry.generate_tools_schema()

### Build the prompt

In [31]:
TOOL_PROMPT = """
You are a function calling AI model.
If a function or tool is unavailable, responde with trained data.
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags, here are the available tools:
<tools>
%s
</tools>

For each function call, to a listed function, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>, "id": <monotonically-increasing-id>}
</tool_call>
""" % tools_schemas_string

In [32]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_PROMPT,
    }
]

agent_chat_history = []

USER_PROMPT = "Can you multiple 25 by 12 and sum 3 with 6 and search what is an LLM on google.com?"

tool_chat_history.append(
    {"role": "user", "content": USER_PROMPT}
)
agent_chat_history.append(
    {"role": "user", "content": USER_PROMPT}
)

# Request tools action
tools_calls_request = client.chat(model=model_name, messages=tool_chat_history)

# Parse the response into model default format
tools_calls_str = tools_calls_request['message']['content']

processor = ToolCallProcessor(tool_registry)
output = processor.process_tool_calls(tools_calls_str)

# Return the final response using the tools retrivied info
agent_chat_history.append({
    "role": "tool",
    "content": output
})

tool_response = client.chat(model=model_name, messages=agent_chat_history)

response_using_tool = tool_response['message']['content']

display_markdown(response_using_tool, raw=True)

Sure, the multiplication of 25 by 12 equals 300, and the sum of 3 with 6 equals 9. 

To answer your last request: LLM stands for Large Language Model. It is a type of artificial intelligence model designed to understand and generate human-like text. These models are typically trained on large amounts of text data and can perform various language-related tasks such as translation, summarization, question answering, and more. If you'd like me to search "what is an LLM" for you using Google, I can provide a brief summary based on the information available, but I won't actually perform the web search itself. Would you like to proceed with that?